In [2]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [3]:
phase = 'apo'
name = 'asn'
state = 0
length = 1
move_length = 1
timestep = 4
radius = 0.3

In [4]:
# Load rhtf 
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/36/"
i = os.path.basename(os.path.dirname(dir))
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf = pickle.load(open(path, "rb" ))



INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/36/36_apo_0.pickle


In [ ]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
# query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(args.resid)].atoms]
for res in htf.hybrid_topology.residues:
    if res.resSeq == 501 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system


In [ ]:
# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))


In [32]:
# from openmmtools.multistate import ReplicaExchangeSampler
# import mpiplus
# from openmmtools import states
# class ReplicaExchangeSampler2(ReplicaExchangeSampler):
    
#     def __init__(self, replica_mixing_scheme='swap-all', reporter2=None, **kwargs):

#         # Initialize multi-state sampler simulation.
#         super(ReplicaExchangeSampler, self).__init__(**kwargs)
#         self.replica_mixing_scheme = replica_mixing_scheme
#         self._reporter_2 = reporter2

    
#     @mpiplus.on_single_node(rank=0, broadcast_result=False, sync_nodes=False)
#     @mpiplus.delayed_termination
#     def _report_iteration_items(self):
#         """
#         Sub-function of :func:`_report_iteration` which handles all the actual individual item reporting in a
#         sub-class friendly way. The final actions of writing timestamp, last-good-iteration, and syncing
#         should be left to the :func:`_report_iteration` and subclasses should extend this function instead
#         """
#         replica_id = np.where(self._replica_thermodynamic_states == 0)[0][0]
#         print("ITERATION: ", self._iteration)
#         print("REPLICA THERMOSTATES ", self._replica_thermodynamic_states, type(self._replica_thermodynamic_states))
#         print("REPLICA ID ", replica_id, type(replica_id))
#         self._reporter.write_sampler_states([self._sampler_states[replica_id]], self._iteration)
#         self._reporter_2.write_sampler_states2(self._sampler_states, self._iteration)
        
#         self._reporter.write_replica_thermodynamic_states(self._replica_thermodynamic_states, self._iteration)
#         self._reporter.write_mcmc_moves(self._mcmc_moves)  # MCMCMoves can store internal statistics.
#         self._reporter.write_energies(self._energy_thermodynamic_states, self._neighborhoods, self._energy_unsampled_states,
#                                       self._iteration)
#         self._reporter.write_mixing_statistics(self._n_accepted_matrix, self._n_proposed_matrix, self._iteration)

#     def create(self, thermodynamic_states: list, sampler_states, storage, storage_2,
#                initial_thermodynamic_states=None, unsampled_thermodynamic_states=None,
#                metadata=None):
#         """Create new multistate sampler simulation.
#         Parameters
#         ----------
#         thermodynamic_states : list of states.ThermodynamicState
#             Thermodynamic states to simulate, where one replica is allocated per state.
#             Each state must have a system with the same number of atoms.
#         sampler_states : states.SamplerState or list
#             One or more sets of initial sampler states.
#             The number of replicas is taken to be the number of sampler states provided.
#             If the sampler states do not have box_vectors attached and the system is periodic,
#             an exception will be thrown.
#         storage : str or instanced Reporter
#             If str: the path to the storage file. Default checkpoint options from Reporter class are used
#             If Reporter: Uses the reporter options and storage path
#             In the future this will be able to take a Storage class as well.
#         initial_thermodynamic_states : None or list or array-like of int of length len(sampler_states), optional,
#             default: None.
#             Initial thermodynamic_state index for each sampler_state.
#             If no initial distribution is chosen, ``sampler_states`` are distributed between the
#             ``thermodynamic_states`` following these rules:
#                 * If ``len(thermodynamic_states) == len(sampler_states)``: 1-to-1 distribution
#                 * If ``len(thermodynamic_states) > len(sampler_states)``: First and last state distributed first
#                   remaining ``sampler_states`` spaced evenly by index until ``sampler_states`` are depleted.
#                   If there is only one ``sampler_state``, then the only first ``thermodynamic_state`` will be chosen
#                 * If ``len(thermodynamic_states) < len(sampler_states)``, each ``thermodynamic_state`` receives an
#                   equal number of ``sampler_states`` until there are insufficient number of ``sampler_states`` remaining
#                   to give each ``thermodynamic_state`` an equal number. Then the rules from the previous point are
#                   followed.
#         unsampled_thermodynamic_states : list of states.ThermodynamicState, optional, default=None
#             These are ThermodynamicStates that are not propagated, but their
#             reduced potential is computed at each iteration for each replica.
#             These energy can be used as data for reweighting schemes (default
#             is None).
#         metadata : dict, optional, default=None
#            Simulation metadata to be stored in the file.
#         """
#         # Handle case in which storage is a string and not a Reporter object.
#         self._reporter = self._reporter_from_storage(storage, check_exist=False)

#         # Check if netcdf files exist. This is run only on MPI node 0 and
#         # broadcasted. This is to avoid the case where the other nodes
#         # arrive to this line after node 0 has already created the storage
#         # file, causing an error.
#         if mpiplus.run_single_node(0, self._reporter.storage_exists, broadcast_result=True):
#             raise RuntimeError('Storage file {} already exists; cowardly '
#                                'refusing to overwrite.'.format(self._reporter.filepath))

#         # Make sure sampler_states is an iterable of SamplerStates.
#         if isinstance(sampler_states, states.SamplerState):
#             sampler_states = [sampler_states]

#         # Initialize internal attribute and dataset.
#         self._pre_write_create(thermodynamic_states, sampler_states, storage,
#                                initial_thermodynamic_states=initial_thermodynamic_states,
#                                unsampled_thermodynamic_states=unsampled_thermodynamic_states,
#                                metadata=metadata)

#         # Display papers to be cited.
#         self._display_citations()

#         self._initialize_reporter()
        
#         # Also initialize second reporter
#         self._reporter_2 = storage2
#         self._reporter_2.open(mode='w')
#         self._reporter_2.write_thermodynamic_states(self._thermodynamic_states,
#                                                   self._unsampled_states)

#         # Store run metadata and ReplicaExchange options.
# #         self._store_options()
# #         self._reporter_2.write_dict('metadata', self._metadata)

#         # Store initial conditions. This forces the storage to be synchronized.
#         self._report_iteration()
        
        
# class MultiStateReporter2(multistate.MultiStateReporter):
    
#     def write_sampler_states2(self, sampler_states: list, iteration: int):
#         """Store all sampler states for a given iteration on the checkpoint file
#         If the iteration is not on the checkpoint interval, only the ``analysis_particle_indices`` data is written,
#         if set.
#         Parameters
#         ----------
#         sampler_states : list of SamplerStates
#             The sampler states to store for each replica.
#         iteration : int
#             The iteration at which to store the data.
#         """
#         print("GOT INTO SUBCLASS WRITE_SAMPLER_STATES2")
        
#         # Case of no special atoms, write to normal checkpoint
#         self._write_sampler_states_to_given_file2(sampler_states, iteration, storage_file='checkpoint',
#                                                  obey_checkpoint_interval=True)
# #         if len(self._analysis_particle_indices) > 0:
# #             # Special case, pre-process the sampler_states
# #             sampler_subset = []
# #             for sampler_state in sampler_states:
# #                 positions = sampler_state.positions
# #                 # Subset positions
# #                 # Need the [arg, :] to get uniform behavior with tuple and list for arg
# #                 # since a ndarray[tuple] is different than ndarray[list]
# #                 position_subset = positions[self._analysis_particle_indices, :]
# #                 sampler_subset.append(states.SamplerState(position_subset,
# #                                                                   box_vectors=sampler_state.box_vectors))
# #             self._write_sampler_states_to_given_file(sampler_subset, iteration, storage_file='analysis',
# #                                                      obey_checkpoint_interval=False)
    
#     def _write_sampler_states_to_given_file2(self, sampler_states: list, iteration: int,
#                                             storage_file='checkpoint', obey_checkpoint_interval=True):
#         """
#         Internal function to handle writing sampler states more generically to target file
#         Parameters
#         ----------
#         sampler_states : list of states.SamplerStates
#             The sampler states to store for each replica.
#         iteration : int
#             The iteration at which to store the data.
#         storage_file : string, Optional, Default: 'checkpoint'
#             Name of storage file we're writing to. Must match a valid key of self._storage_dict
#         obey_checkpoint_interval : bool, Optional, Default: False
#             Tells this (attempted) write to obey the checkpoint interval or not.
#             If True, no write out will be done if iteration is not on the checkpoint interval
#             If False, the write WILL occur
#         """
#         print("GOT INTO SUBCLASS WRITE_SAMPLER_STATES_TO_GIVEN_FILE2")

#         storage = "/data/chodera/zhangi/perses_benchmark/neq/14/36/test/all_checkpoint.nc"
#         # Check if the schema must be initialized, do this regardless of the checkpoint_interval for consistency
#         is_periodic = True if (sampler_states[0].box_vectors is not None) else False
#         n_particles = sampler_states[0].n_particles
#         n_replicas = len(sampler_states)
#         self._initialize_sampler_variables_on_file(storage, n_particles,
#                                                    n_replicas, is_periodic)
#         if obey_checkpoint_interval:
#             write_iteration = self._calculate_checkpoint_iteration(iteration)
#         else:
#             write_iteration = iteration
#         # Write the sampler state if we are on the checkpoint interval OR if told to ignore the interval
#         if write_iteration is not None:
#             # Store sampler states.
#             # Create a numpy array to avoid making multiple (possibly inefficient) calls to netCDF assignments
#             positions = np.zeros([n_replicas, n_particles, 3])
#             for replica_index, sampler_state in enumerate(sampler_states):
#                 # Store positions in memory first
#                 x = sampler_state.positions / unit.nanometers
#                 positions[replica_index, :, :] = x[:, :]
#             # Store positions
#             storage.variables['positions'][write_iteration, :, :, :] = positions

#             if is_periodic:
#                 # Store box vectors and volume.
#                 # Allocate whole write to memory first
#                 box_vectors = np.zeros([n_replicas, 3, 3])
#                 volumes = np.zeros([n_replicas])
#                 for replica_index, sampler_state in enumerate(sampler_states):
#                     box_vectors[replica_index, :, :] = sampler_state.box_vectors / unit.nanometers
#                     volumes[replica_index] = sampler_state.volume / unit.nanometers ** 3
#                 storage.variables['box_vectors'][write_iteration, :, :, :] = box_vectors
#                 storage.variables['volumes'][write_iteration, :] = volumes

#         else:
#             logger.debug("Iteration {} not on the Checkpoint Interval of {}. "
#                          "Sampler State not written.".format(iteration, self._checkpoint_interval))
        
        

In [49]:
from openmmtools.multistate import ReplicaExchangeSampler
import mpiplus
from openmmtools import states
class ReplicaExchangeSampler2(ReplicaExchangeSampler):
    
    @mpiplus.on_single_node(rank=0, broadcast_result=False, sync_nodes=False)
    @mpiplus.delayed_termination
    def _report_iteration_items(self):
        """
        Sub-function of :func:`_report_iteration` which handles all the actual individual item reporting in a
        sub-class friendly way. The final actions of writing timestamp, last-good-iteration, and syncing
        should be left to the :func:`_report_iteration` and subclasses should extend this function instead
        """
        replica_id = np.where(self._replica_thermodynamic_states == 0)[0][0]
        print("ITERATION: ", self._iteration)
        print("REPLICA THERMOSTATES ", self._replica_thermodynamic_states, type(self._replica_thermodynamic_states))
        print("REPLICA ID ", replica_id, type(replica_id))
        self._reporter.write_sampler_states([self._sampler_states[replica_id]], self._iteration)
        self._reporter.write_sampler_states2(self._sampler_states, self._iteration)
        
        self._reporter.write_replica_thermodynamic_states(self._replica_thermodynamic_states, self._iteration)
        self._reporter.write_mcmc_moves(self._mcmc_moves)  # MCMCMoves can store internal statistics.
        self._reporter.write_energies(self._energy_thermodynamic_states, self._neighborhoods, self._energy_unsampled_states,
                                      self._iteration)
        self._reporter.write_mixing_statistics(self._n_accepted_matrix, self._n_proposed_matrix, self._iteration)
        
class MultiStateReporter2(multistate.MultiStateReporter):
    
    def write_sampler_states2(self, sampler_states: list, iteration: int):
        """Store all sampler states for a given iteration on the checkpoint file
        If the iteration is not on the checkpoint interval, only the ``analysis_particle_indices`` data is written,
        if set.
        Parameters
        ----------
        sampler_states : list of SamplerStates
            The sampler states to store for each replica.
        iteration : int
            The iteration at which to store the data.
        """
        print("GOT INTO SUBCLASS WRITE_SAMPLER_STATES2")
        
        # Case of no special atoms, write to normal checkpoint
        self._write_sampler_states_to_given_file2(sampler_states, iteration, storage_file='checkpoint',
                                                 obey_checkpoint_interval=True)
#         if len(self._analysis_particle_indices) > 0:
#             # Special case, pre-process the sampler_states
#             sampler_subset = []
#             for sampler_state in sampler_states:
#                 positions = sampler_state.positions
#                 # Subset positions
#                 # Need the [arg, :] to get uniform behavior with tuple and list for arg
#                 # since a ndarray[tuple] is different than ndarray[list]
#                 position_subset = positions[self._analysis_particle_indices, :]
#                 sampler_subset.append(states.SamplerState(position_subset,
#                                                                   box_vectors=sampler_state.box_vectors))
#             self._write_sampler_states_to_given_file(sampler_subset, iteration, storage_file='analysis',
#                                                      obey_checkpoint_interval=False)
    
    def _write_sampler_states_to_given_file2(self, sampler_states: list, iteration: int,
                                            storage_file='checkpoint', obey_checkpoint_interval=True):
        """
        Internal function to handle writing sampler states more generically to target file
        Parameters
        ----------
        sampler_states : list of states.SamplerStates
            The sampler states to store for each replica.
        iteration : int
            The iteration at which to store the data.
        storage_file : string, Optional, Default: 'checkpoint'
            Name of storage file we're writing to. Must match a valid key of self._storage_dict
        obey_checkpoint_interval : bool, Optional, Default: False
            Tells this (attempted) write to obey the checkpoint interval or not.
            If True, no write out will be done if iteration is not on the checkpoint interval
            If False, the write WILL occur
        """
        print("GOT INTO SUBCLASS WRITE_SAMPLER_STATES_TO_GIVEN_FILE2")

        storage = "/data/chodera/zhangi/perses_benchmark/neq/14/36/test/all_checkpoint.nc"
        # Check if the schema must be initialized, do this regardless of the checkpoint_interval for consistency
        is_periodic = True if (sampler_states[0].box_vectors is not None) else False
        n_particles = sampler_states[0].n_particles
        n_replicas = len(sampler_states)

        # Store sampler states.
        # Create a numpy array to avoid making multiple (possibly inefficient) calls to netCDF assignments
        positions = np.zeros([n_replicas, n_particles, 3])
        for replica_index, sampler_state in enumerate(sampler_states):
            # Store positions in memory first
            x = sampler_state.positions / unit.nanometers
            positions[replica_index, :, :] = x[:, :]
        
        with open(f"/data/chodera/zhangi/perses_benchmark/neq/14/36/test/{iteration}.npy", "wb") as f:
            np.save(f, positions)
        
        print("Finished writing")
           

In [50]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))

# Run t-repex
reporter_file = os.path.join(dir, "test/" f"{i}_{phase}_{name.lower()}_{length}ns.nc")
reporter = MultiStateReporter2(reporter_file, checkpoint_interval=1)
# reporter2 = MultiStateReporter2(reporter_file, checkpoint_interval=1)
simulation = ReplicaExchangeSampler2(mcmc_moves=move, number_of_iterations=length*1000)

simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()


INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/14/36/test/36_apo_asn_1ns_checkpoint.nc
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <__main__.MultiStateReporter2 object at 0x2b0a1436ef70>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b09bdcd1430>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b09bdc88d30>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b09bdc8a040>


move steps: 250
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  664339B | 648.769KB | 0.634MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.939s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b09bdc8a160>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler2._report_iteration_items at 0x2b0a17e05b80>
DEBUG:openmmtools.utils:Storing sampler states took    0.004s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.059s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.


ITERATION:  0
REPLICA THERMOSTATES  [ 0  1  2  3  4  5  6  7  8  9 10 11] <class 'numpy.ndarray'>
REPLICA ID  0 <class 'numpy.int64'>
GOT INTO SUBCLASS WRITE_SAMPLER_STATES2
GOT INTO SUBCLASS WRITE_SAMPLER_STATES_TO_GIVEN_FILE2
Finished writing


DEBUG:openmmtools.utils:Computing energy matrix took    0.170s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <__main__.MultiStateReporter2 object at 0x2b0a1436ef70>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b09bdcd10d0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.002s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 13146/41472 attempted swaps (31.7%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:m

...

DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    3.520s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.139s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b09bdc8a160>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler2._report_iteration_items at 0x2b0a17e05b80>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.043s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b09bdc8af70>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multist

ITERATION:  11
REPLICA THERMOSTATES  [ 1  2  0  3  5  8  6  7  9 11 10  4] <class 'numpy.ndarray'>
REPLICA ID  2 <class 'numpy.int64'>
GOT INTO SUBCLASS WRITE_SAMPLER_STATES2
GOT INTO SUBCLASS WRITE_SAMPLER_STATES_TO_GIVEN_FILE2
Finished writing


DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    3.575s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.165s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b09bdc8a160>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler2._report_iteration_items at 0x2b0a17e05b80>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.053s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b09bdc8af70>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multist

ITERATION:  12
REPLICA THERMOSTATES  [ 1  2  0  3  5  8  7  6 10 11  9  4] <class 'numpy.ndarray'>
REPLICA ID  2 <class 'numpy.int64'>
GOT INTO SUBCLASS WRITE_SAMPLER_STATES2
GOT INTO SUBCLASS WRITE_SAMPLER_STATES_TO_GIVEN_FILE2
Finished writing


KeyboardInterrupt: 

In [8]:
# Load positions from nc

new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)

from perses.analysis.utils import open_netcdf
nc = open_netcdf(os.path.join(dir, "test", f"{i}_{phase}_{name.lower()}_{length}ns.nc"))
nc_checkpoint = open_netcdf(os.path.join(dir, "test", f"{i}_{phase}_{name.lower()}_{length}ns_checkpoint.nc"))
checkpoint_interval = nc_checkpoint.CheckpointInterval
all_positions = nc_checkpoint.variables['positions']
n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)

from tqdm import tqdm
all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
all_pos_hybrid = np.zeros(shape=(n_iter, n_atoms, 3))
for iteration in tqdm(range(n_iter)):
    # replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == 0)[0]
    replica_id = 0
    pos = all_positions[iteration,replica_id,:,:] *unit.nanometers
    all_pos_hybrid[iteration] = pos # Get hybrid positions


100%|██████████| 13/13 [00:00<00:00, 427.69it/s]


In [13]:
# Load positions from npys
all_pos_hybrid_npys = np.zeros(shape=(n_iter, n_atoms, 3))
for i in range(13):
    replica_id = np.where(nc.variables['states'][i*checkpoint_interval] == 0)[0][0]
    print(replica_id)
    with open(os.path.join(dir, "test", f"{i}.npy"), "rb") as f:
        arr = np.load(f)
    all_pos_hybrid_npys[i] = arr[replica_id]

0
3
3
3
3
1
1
2
2
2
2
2
2


In [9]:
all_pos_hybrid.shape

(13, 44758, 3)

In [17]:
for i, (pos_nc, pos_npy) in enumerate(zip(all_pos_hybrid, all_pos_hybrid_npys)):
    print(i, np.allclose(pos_nc, pos_npy))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True


In [18]:
pos_nc[0]

array([1.9170047 , 3.29341054, 4.14650774])

In [19]:
pos_npy[0]

array([1.91700465, 3.29341065, 4.14650752])